In [6]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [7]:
%reload_ext kedro.ipython

[05/16/24 21:38:13] INFO     Registered line magic '%reload_kedro'                                   ]8;id=414255;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=813528;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#51\51]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=752191;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=384193;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#53\53]8;;\

                    INFO     Resolved project path as: /Users/gavinlou/Developer/gasai.             ]8;id=675022;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=646641;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#164\164]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/16/24 21:38:14] INFO     Kedro project gasAI                                                    ]8;id=716357;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=920363;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=694047;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=636315;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#135\135]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=980724;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=79230;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#141\141]8;;\

In [8]:
df = catalog.load("s3_conc_aligned_df")

                    INFO     Loading data from s3_conc_aligned_df (ParquetDataset)...           ]8;id=731500;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=56616;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

class GroupDataset(Dataset):
    def __init__(self, X, y, groups):
        self.X = X.unsqueeze(2)
        self.y = y.unsqueeze(1)
        self.groups = groups
        self.group_to_indices = self._group_indices()

    def _group_indices(self):
        group_to_indices = {}
        for idx, group in enumerate(self.groups):
            if group not in group_to_indices:
                group_to_indices[group] = []
            group_to_indices[group].append(idx)
        return group_to_indices

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.groups[idx]

class ExperimentSampler:
    def __init__(self, group_to_indices, batch_size):
        self.group_to_indices = group_to_indices
        self.batch_size = batch_size
        self.group_order = list(group_to_indices.keys())
        np.random.shuffle(self.group_order)  # Shuffle the order of groups

    def __iter__(self):
        for group in self.group_order:
            indices = self.group_to_indices[group]
            for i in range(0, len(indices), self.batch_size):
                yield indices[i:i + self.batch_size]

    def __len__(self):
        total_batches = 0
        for indices in self.group_to_indices.values():
            total_batches += (len(indices) + self.batch_size - 1) // self.batch_size
        return total_batches

# Define the features and target
features = ['timestamp_bin', 'A1_Resistance']
target = 'resistance_ratio'
X_tensor = df[features]
y_tensor = df[target]
groups = df['exp_no']
batch_size = 128

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_tensor.values, dtype=torch.float32)
y_tensor = torch.tensor(y_tensor.values, dtype=torch.float32)
# Convert groups to NumPy array
groups = np.array(groups)

# Create dataset
# dataset = GroupDataset(X_tensor, y_tensor, groups)


# Assuming X_tensor and y_tensor are your features and targets respectively

# 1. Split the data into training and testing sets using GroupShuffleSplit
gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idx, test_idx = next(gss.split(X_tensor, y_tensor, groups=groups))

# 2. Fit the scaler on the training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_tensor[train_idx].numpy())  # Convert to NumPy if necessary and scale

# 3. Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_tensor[test_idx].numpy())

# Convert scaled data back to tensors
X_train_scaled_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_scaled_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# 4. Create datasets
train_dataset = GroupDataset(X_train_scaled_tensor, y_tensor[train_idx], groups[train_idx])
test_dataset = GroupDataset(X_test_scaled_tensor, y_tensor[test_idx], groups[test_idx])

# 5. Create experiment samplers
train_sampler = ExperimentSampler(train_dataset.group_to_indices, batch_size)
test_sampler = ExperimentSampler(test_dataset.group_to_indices, batch_size)

# 6. Create data loaders
train_loader = DataLoader(train_dataset, batch_sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_sampler=test_sampler)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3, padding=1)
        # self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 16)  # Adjust the input features depending on the output size after conv and pool layers
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [ ]:
from torch.optim import Adam
from math import sqrt


device = torch.device("mps" if torch.cuda.is_available() else "cpu")

model = CNNModel().to(device)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

def calculate_rmse(outputs, labels):
    mse = torch.mean((outputs - labels) ** 2)  # Calculate MSE as a tensor
    return torch.sqrt(mse)  # Return RMSE as a tensor



def train_model(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels, _ in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            # inputs = inputs.transpose(1, 2)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        model.eval()

        # During validation
        val_rmse = 0.0
        total_samples = 0
        with torch.no_grad():
            for inputs, labels, _ in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_rmse = calculate_rmse(outputs, labels) * inputs.size(0)
                val_rmse += batch_rmse.item()  # Properly use .item() to convert tensor to float
                total_samples += inputs.size(0)

        val_rmse /= total_samples
        print(f'Epoch {epoch+1} Train Loss: {epoch_loss:.4f} Val RMSE: {val_rmse:.4f}')

# Training the model
train_model(model, train_loader, test_loader, n_epochs=15)


Epoch 1 Train Loss: 0.0495 Val RMSE: 0.0847
Epoch 2 Train Loss: 0.0126 Val RMSE: 0.0777
Epoch 3 Train Loss: 0.0108 Val RMSE: 0.0637
Epoch 4 Train Loss: 0.0093 Val RMSE: 0.0594
Epoch 5 Train Loss: 0.0084 Val RMSE: 0.0570
Epoch 6 Train Loss: 0.0078 Val RMSE: 0.0569
Epoch 7 Train Loss: 0.0074 Val RMSE: 0.0563
Epoch 8 Train Loss: 0.0070 Val RMSE: 0.0555
Epoch 9 Train Loss: 0.0067 Val RMSE: 0.0528
Epoch 10 Train Loss: 0.0065 Val RMSE: 0.0532
Epoch 11 Train Loss: 0.0063 Val RMSE: 0.0582
Epoch 12 Train Loss: 0.0061 Val RMSE: 0.0601
Epoch 13 Train Loss: 0.0058 Val RMSE: 0.0592
Epoch 14 Train Loss: 0.0057 Val RMSE: 0.0580
Epoch 15 Train Loss: 0.0056 Val RMSE: 0.0563


In [ ]:
for inputs, label, group in train_loader:
    print(inputs)
    break
